# CME Most liquid contracts

In [264]:
import pandas as pd

- Eurex
    - https://www.eurex.com/ex-en/data/statistics/trading-statistics (all products)

- CME
    - https://www.cmegroup.com/CmeWS/exp/voiProductsViewExport.ctl?media=xls&tradeDate=20220210&assetClassId=2&reportType=F&excluded=CEE,CEU,KCB (Agriculture)
    - https://www.cmegroup.com/CmeWS/exp/voiProductsViewExport.ctl?media=xls&tradeDate=20220210&assetClassId=7&reportType=F&excluded=CEE,CEU,KCB (Energy)
    - https://www.cmegroup.com/CmeWS/exp/voiProductsViewExport.ctl?media=xls&tradeDate=20220210&assetClassId=4&reportType=F&excluded=CEE,CEU,KCB (Equities)
    - https://www.cmegroup.com/CmeWS/exp/voiProductsViewExport.ctl?media=xls&tradeDate=20220210&assetClassId=3&reportType=F&excluded=CEE,CEU,KCB (FX)
    - https://www.cmegroup.com/CmeWS/exp/voiProductsViewExport.ctl?media=xls&tradeDate=20220210&assetClassId=6&reportType=F&excluded=CEE,CEU,KCB (Interest Rate)
    - https://www.cmegroup.com/CmeWS/exp/voiProductsViewExport.ctl?media=xls&tradeDate=20220210&assetClassId=8&reportType=F&excluded=CEE,CEU,KCB (Metals)
    
- ICE
    - https://www.theice.com/publicdocs/Vendor_Codes.xlsx (code list only, all products ICE US + ICE Europe)

## Load data

### CME

In [268]:
volumes_Agriculture = pd.read_excel('/mnt/c/Users/fabie/Downloads/VoiTotalsByAssetClassExcelExport_Agriculture.xlsx', skiprows=4, thousands=',')
volumes_Energy = pd.read_excel('/mnt/c/Users/fabie/Downloads/VoiTotalsByAssetClassExcelExport_Energy.xls', skiprows=4, thousands=',')
volumes_Metals = pd.read_excel('/mnt/c/Users/fabie/Downloads/VoiTotalsByAssetClassExcelExport_Metals.xls', skiprows=4, thousands=',')
volumes_Fx = pd.read_excel('/mnt/c/Users/fabie/Downloads/VoiTotalsByAssetClassExcelExport_FX.xls', skiprows=4, thousands=',')
volumes_InterestRate = pd.read_excel('/mnt/c/Users/fabie/Downloads/VoiTotalsByAssetClassExcelExport_InterestRate.xls', skiprows=4, thousands=',')
volumes_Equities = pd.read_excel('/mnt/c/Users/fabie/Downloads/VoiTotalsByAssetClassExcelExport_Equities.xls', skiprows=4, thousands=',')

In [269]:
volumes_Agriculture = volumes_Agriculture[volumes_Agriculture.Type == 'Futures'].reset_index(drop=True)
volumes_Energy = volumes_Energy[volumes_Energy.Type == 'Futures'].reset_index(drop=True)
volumes_Metals = volumes_Metals[volumes_Metals.Type == 'Futures'].reset_index(drop=True)
volumes_Fx = volumes_Fx[volumes_Fx.Type == 'Futures'].reset_index(drop=True)
volumes_InterestRate = volumes_InterestRate[volumes_InterestRate.Type == 'Futures'].reset_index(drop=True)
volumes_Equities = volumes_Equities[volumes_Equities.Type == 'Futures'].reset_index(drop=True)

### Eurex

In [272]:
# Very hugly
def parse_eurex_dailystat(path):
    xls = pd.ExcelFile(path)
    df1 = pd.read_excel(xls, "Eurex Daily Statistics", skiprows=10, thousands=' ')

    future_filter = (df1['Unnamed: 2'].str.endswith('Futures').fillna(False)) & ~(df1['Unnamed: 2'].str.startswith('Options on').fillna(False))
    futures_sections_begin_index = df1[future_filter]

    all_futs = []
    for begin_index, cat_name in futures_sections_begin_index['Unnamed: 2'].items():
        sliced_df = df1.iloc[begin_index:, :].reset_index(drop=True)
        end_indx = sliced_df[sliced_df['Unnamed: 3'].str.startswith('Sum').fillna(False)].head(n=1).index.values[0]
        sliced_df = sliced_df.iloc[:end_indx-1]
        sliced_df['Unnamed: 2'] = cat_name
        all_futs.append(sliced_df)

    all_eurex_futs = pd.concat(all_futs)
    all_eurex_futs = all_eurex_futs.dropna(how='all', subset=['Unnamed: 3'])
    all_eurex_futs = all_eurex_futs.dropna(how='all', subset=list(all_eurex_futs.columns[5:]))

    for column in ('Volume in EUR', 'Volume in EUR (month)', 'Volume in EUR (year)', 
                   'Traded Contracts', 'Traded Contracts (month)', 'Traded Contracts (year)'):
        all_eurex_futs[column] = all_eurex_futs[column].astype(float)
        

    all_eurex_futs = all_eurex_futs.sort_values(by='Volume in EUR', ascending=False)
    all_eurex_futs = all_eurex_futs.iloc[:, 2:]
    return all_eurex_futs.rename(columns={'Unnamed: 2': 'Category', 'Unnamed: 3': 'Product', 'Unnamed: 4': 'Code'})

In [275]:
all_eurex_futures = parse_eurex_dailystat('/mnt/c/Users/fabie/Downloads/dailystat_20220210.xls')

### ICE

In [287]:
all_ice_products = pd.read_excel('/mnt/c/Users/fabie/Downloads/Vendor_Codes.xlsx', skiprows=2)
all_ice_futures = all_ice_products[all_ice_products['Contract Type'] == 'Futures']

In [291]:
all_ice_futures

,Exchange,Contract Type,Contract,Period,Level I,Level II,Combined Code
1,ICE Futures Europe,Futures,Brent Crude,months,CB,CB,---
2,ICE Futures Europe,Futures,Brent Crude,month to month spreads,NaN,NaN,---
3,ICE Futures Europe,Futures,Brent Crude,settlement contract,NaN,NaN,---
4,ICE Futures Europe,Futures,Brent Crude,quarters,NaN,NaN,---
5,ICE Futures Europe,Futures,Brent Crude,quarter to quarter spreads,NaN,NaN,---
...,...,...,...,...,...,...,...
272,ICE Futures U.S.,Futures,Canadian dollar/Japenese Yen,month to month spreads,NaN,NaN,NaN
273,ICE Futures U.S.,Futures,Colombian Peso - USD,months,NaN,NaN,NaN
274,ICE Futures U.S.,Futures,Colombian Peso - USD,month to month spreads,NaN,NaN,NaN
276,ICE Futures U.S.,Futures,Brazilian Real - US$,months,NaN,NaN,NaN


# Agriculture

In [45]:
volumes_Agriculture.sort_values(by='Volume', ascending=False).head(n=20)

,Name,Type,Globex,Open OutCry,Clear Port,Volume,Open Interest,Change
8,Corn Futures,Futures,750698.0,0.0,14858.0,765556.0,1602289.0,29320.0
24,Soybean Futures,Futures,525095.0,0.0,12798.0,537893.0,863094.0,11673.0
25,Soybean Meal Futures,Futures,237219.0,0.0,4886.0,242105.0,440844.0,3521.0
5,Chicago SRW Wheat Futures,Futures,197191.0,0.0,63.0,197254.0,397153.0,5473.0
26,Soybean Oil Futures,Futures,159602.0,0.0,2650.0,162252.0,424929.0,-3736.0
13,KC HRW Wheat Futures,Futures,82350.0,0.0,1766.0,84116.0,214929.0,2641.0
15,Live Cattle Futures,Futures,58290.0,0.0,0.0,58290.0,350877.0,1427.0
14,Lean Hog Futures,Futures,53980.0,0.0,90.0,54070.0,278867.0,686.0
11,Feeder Cattle Futures,Futures,16383.0,0.0,0.0,16383.0,46680.0,46.0
16,Mini Soybean Futures,Futures,8252.0,0.0,0.0,8252.0,29969.0,1246.0


# Energy

In [41]:
volumes_Energy.sort_values(by='Volume', ascending=False).head(n=20)

,Name,Type,Globex,Open OutCry,Clear Port,Volume,Open Interest,Change
34,Crude Oil Futures,Futures,1213054.0,0.0,21303.0,1234357.0,2175221.0,4590.0
94,Henry Hub Natural Gas Futures,Futures,457298.0,0.0,12477.0,469775.0,1096101.0,-9010.0
189,RBOB Gasoline Futures,Futures,248839.0,0.0,17438.0,266277.0,395858.0,-3799.0
167,NY Harbor ULSD Futures,Futures,169969.0,0.0,7077.0,177046.0,362937.0,4582.0
17,Brent Last Day Financial Futures,Futures,102562.0,0.0,0.0,102562.0,210804.0,2042.0
122,Micro WTI Crude Oil Futures,Futures,83650.0,0.0,0.0,83650.0,13298.0,1804.0
40,E-mini Crude Oil Futures,Futures,26643.0,0.0,0.0,26643.0,5178.0,192.0
155,Natural Gas (Henry Hub) Last-day Financial Fut...,Futures,11203.0,0.0,5210.0,16413.0,479228.0,238.0
147,Mont Belvieu LDH Propane (OPIS) Futures,Futures,0.0,0.0,11361.0,11361.0,124003.0,1631.0
206,WTI Financial Futures,Futures,0.0,0.0,6733.0,6733.0,151676.0,1922.0


# Metals

In [46]:
volumes_Metals.sort_values(by='Volume', ascending=False).head(n=20)

,Name,Type,Globex,Open OutCry,Clear Port,Volume,Open Interest,Change
12,Gold Futures,Futures,247386.0,0.0,2802.0,250188.0,530507.0,9881.0
8,Copper Futures,Futures,180599.0,0.0,150.0,180749.0,221429.0,7111.0
23,Silver Futures,Futures,102654.0,0.0,806.0,103460.0,156516.0,5993.0
16,Micro Gold Futures,Futures,43553.0,0.0,0.0,43553.0,24329.0,225.0
20,Platinum Futures,Futures,22583.0,0.0,0.0,22583.0,59500.0,479.0
17,Micro Silver Futures,Futures,9033.0,0.0,0.0,9033.0,6134.0,-39.0
19,Palladium Futures,Futures,2594.0,0.0,0.0,2594.0,7733.0,140.0
25,U.S. Midwest Domestic Hot-Rolled Coil Steel (C...,Futures,375.0,0.0,902.0,1277.0,39415.0,477.0
22,Shanghai Gold (USD) Futures,Futures,1169.0,0.0,0.0,1169.0,735.0,62.0
10,E-mini Gold Futures,Futures,770.0,0.0,0.0,770.0,1598.0,-43.0


# Fx

In [47]:
volumes_Fx.sort_values(by='Volume', ascending=False).head(n=20)

,Name,Type,Globex,Open OutCry,Clear Port,Volume,Open Interest,Change
14,Euro FX Futures,Futures,271533.0,0.0,1925.0,273458.0,700494.0,0.0
24,Japanese Yen Futures,Futures,166772.0,0.0,7173.0,173945.0,203754.0,4813.0
0,Australian Dollar Futures,Futures,148346.0,0.0,652.0,148998.0,194360.0,-3702.0
5,British Pound Futures,Futures,141093.0,0.0,1023.0,142116.0,202807.0,3696.0
8,Canadian Dollar Futures,Futures,94012.0,0.0,330.0,94342.0,141749.0,-897.0
26,Mexican Peso Futures,Futures,66427.0,0.0,2884.0,69311.0,141209.0,2789.0
37,New Zealand Dollar Futures,Futures,40349.0,0.0,112.0,40461.0,55274.0,-588.0
30,Micro EUR/USD Futures,Futures,39079.0,0.0,0.0,39079.0,37251.0,-1028.0
45,Swiss Franc Futures,Futures,27829.0,0.0,1.0,27830.0,42582.0,169.0
27,Micro AUD/USD Futures,Futures,25516.0,0.0,0.0,25516.0,6816.0,-198.0


# Interest Rate

In [48]:
volumes_InterestRate.sort_values(by='Volume', ascending=False).head(n=20)

,Name,Type,Globex,Open OutCry,Clear Port,Volume,Open Interest,Change
22,Eurodollar Futures,Futures,5929712.0,76423.0,118900.0,6125035.0,11099568.0,92818.0
3,10-Year T-Note Futures,Futures,3109864.0,0.0,96311.0,3206175.0,4165015.0,116158.0
18,5-Year T-Note Futures,Futures,2145063.0,0.0,96084.0,2241147.0,4078970.0,68643.0
32,Three-Month SOFR Futures,Futures,1902053.0,0.0,6000.0,1908053.0,2897916.0,47996.0
12,30 Day Federal Funds Futures,Futures,1472424.0,0.0,5151.0,1477575.0,1968783.0,169074.0
8,2-Year T-Note Futures,Futures,998131.0,0.0,37304.0,1035435.0,2317453.0,-4634.0
34,Ultra 10-Year U.S. Treasury Note Futures,Futures,675077.0,0.0,27661.0,702738.0,1462177.0,20956.0
33,U.S. Treasury Bond Futures,Futures,536318.0,0.0,8544.0,544862.0,1217077.0,23716.0
35,Ultra U.S. Treasury Bond Futures,Futures,284662.0,0.0,16487.0,301149.0,1297322.0,9788.0
29,One-Month SOFR Futures,Futures,161478.0,0.0,0.0,161478.0,338583.0,4321.0


# Equities

In [49]:
volumes_Equities.sort_values(by='Volume', ascending=False).head(n=20)

,Name,Type,Globex,Open OutCry,Clear Port,Volume,Open Interest,Change
31,E-mini S&P 500 Futures,Futures,2372175.0,0.0,37359.0,2409534.0,2220989.0,-2653.0
42,Micro E-mini Nasdaq-100 Index Futures,Futures,1817666.0,0.0,0.0,1817666.0,166263.0,1505.0
44,Micro E-mini S&P 500 Index Futures,Futures,1661642.0,0.0,0.0,1661642.0,174613.0,-668.0
25,E-mini Nasdaq-100 Futures,Futures,826695.0,0.0,10000.0,836695.0,233096.0,4083.0
11,E-mini Russell 2000 Index Futures,Futures,330757.0,0.0,2266.0,333023.0,503970.0,4392.0
16,E-mini Dow ($5) Futures,Futures,282184.0,0.0,900.0,283084.0,81816.0,-981.0
41,Micro E-mini Dow Jones Industrial Average Inde...,Futures,258758.0,0.0,0.0,258758.0,28520.0,-1508.0
43,Micro E-mini Russell 2000 Index Futures,Futures,229982.0,0.0,0.0,229982.0,40624.0,1737.0
51,Nikkei/Yen Futures,Futures,48050.0,0.0,0.0,48050.0,50946.0,-1345.0
40,Micro Bitcoin Futures,Futures,27053.0,0.0,0.0,27053.0,14749.0,1017.0


In [280]:
# All Eurex Products

In [281]:
all_eurex_futures.head(n=20)

,Category,Product,Code,Traded Contracts,Traded Contracts Flexible Products,Traded Contracts (month),Traded Contracts Flexible Products (month),Traded Contracts (year),Traded Contracts Flexible Products (year),Traded Contracts - avg (month),Traded Contracts - avg (year),P/C Ratio,Open Interest (prev Day),Volume in EUR,Volume in EUR (month),Volume in EUR (year),Open Interest in EUR (prev Day)
11,Fixed Income Futures,Euro-Bund Futures,FGBL,1122522.0,NaN,8321849.0,NaN,23178918.0,NaN,1040231.125,799273.034483,NaN,1325628.0,1.851054e+11,1.384078e+12,3.911812e+12,2.197703e+11
8,Fixed Income Futures,Euro-Bobl Futures,FGBM,849148.0,NaN,6345675.0,NaN,15516679.0,NaN,793209.375,535057.896552,NaN,1713754.0,1.107460e+11,8.307077e+11,2.048801e+12,2.240750e+11
14,Fixed Income Futures,Euro-Schatz Futures,FGBS,802862.0,NaN,5146241.0,NaN,11372921.0,NaN,643280.125,392169.689655,NaN,1314861.0,8.954533e+10,5.741318e+11,1.271557e+12,1.467231e+11
13,Fixed Income Futures,Euro-OAT-Futures,FOAT,236217.0,NaN,2090161.0,NaN,5699072.0,NaN,261270.125,196519.724138,NaN,599308.0,3.693864e+10,3.305927e+11,9.157804e+11,9.432183e+10
10,Fixed Income Futures,Euro-BTP Futures,FBTP,239055.0,NaN,1883444.0,NaN,4654889.0,NaN,235430.500,160513.413793,NaN,427213.0,3.333683e+10,2.666473e+11,6.708531e+11,6.015031e+10
4,Equity Index Futures,DAX® Futures,FDAX,78806.0,NaN,555971.0,NaN,2373812.0,NaN,69496.375,81855.586207,NaN,67074.0,3.054131e+10,2.136526e+11,9.257824e+11,2.594495e+10
11,Equity Index Futures,EURO STOXX 50® Index Futures,FESX,728799.0,NaN,6068899.0,NaN,22457245.0,NaN,758612.375,774387.758621,NaN,2999150.0,3.052745e+10,2.516686e+11,9.417925e+11,1.254275e+11
12,Fixed Income Futures,Euro-Buxl® Futures,FGBX,132704.0,NaN,930361.0,NaN,2456645.0,NaN,116295.125,84711.896552,NaN,345361.0,2.590765e+10,1.846788e+11,4.965602e+11,6.793985e+10
45,Equity Index Futures,Mini-DAX Futures,FDXM,53057.0,NaN,372201.0,NaN,1492258.0,NaN,46525.125,51457.172414,NaN,10949.0,4.112470e+09,2.861153e+10,1.162593e+11,8.469270e+08
180,Equity Index Futures,SMI® Futures,FSMI,31117.0,NaN,230692.0,NaN,933964.0,NaN,28836.500,32205.655172,NaN,186722.0,3.590219e+09,2.677568e+10,1.101477e+11,2.172243e+10
